In [1]:
import numpy
import scipy.stats
from scipy.integrate import quad, dblquad

In [2]:
LOW_BOUND = -10.
UPP_BOUND = 10.

In [3]:
N = 3

# Gaussian expectation vector
Mu = numpy.zeros(N)

# Gaussian variance-covariance matrix
rho = 0.
Sigma = numpy.array([[.5, .5*rho, 0.], [.5*rho, .5, 0.], [0., 0., .6]])

print "Mu = %s" % Mu
print "Sigma = \n %s" % Sigma

ctp = 0.

Mu = [ 0.  0.  0.]
Sigma = 
 [[ 0.5  0.   0. ]
 [ 0.   0.5  0. ]
 [ 0.   0.   0.6]]


# Link between paper and code

\begin{equation*}
\ell(x) = \sum_k x_k + \frac{1}{2} \sum_k (x_k^+)^2 + \sum_{j < k} x_j^ + x_k^+
\end{equation*}

We need to solve the following system:
    
\begin{equation*}
\begin{cases}
\lambda \left[ 1
+ \mathbb{E} \left( \left(X_k - m_k^* \right)^+ \right) 
+ \sum_{j \neq k} \mathbb{E} \left( \left(X_j - m_j^* \right)^+ 1_{X_k > m_k^*} \right) 
\right] = 1 \text{ for all $k$} \\
\sum_k \mathbb{E} \left( X_k - m_k^* \right) 
+ \sum_k \frac{1}{2} \mathbb{E} \left( \left[ \left( X_k - m_k^* \right)^+ \right]^2 \right) 
+ \sum_{j < k} \mathbb{E} \left( \left( X_j - m_j^* \right)^+ \left( X_k - m_k^* \right)^+ \right)
= c
\end{cases}
\end{equation*}

Definitions from the paper:

\begin{equation*}
\begin{cases}
\lambda \left[ 1
+ f_k \left( m_k^* \right) 
+ \sum_{j \neq k} l_{j, k} \left( m_j^*, m_k^* \right) 
\right] = 1 \text{ for all $k$} \\
\sum_k \left( e_k \left( m_k^* \right) 
+ \frac{1}{2} g_k \left( m_k^* \right) 
+ \sum_{j < k} h \left( m_j^*, m_k^* \right)
\right) 
= c
\end{cases}
\end{equation*}

In [4]:
def e(index, x):
    mu = Mu[index]
    
    return mu - x

In [5]:
def fourier_integral1d(cplx_integrand, j, x, alpha):
    import warnings
    warnings.filterwarnings('error')
    
    def real_integrand(u):
        return scipy.real(cplx_integrand(u, j, x, alpha))

    real_quad = quad(real_integrand, -numpy.inf, numpy.inf, epsrel=1e-4)[0]
    
    return real_quad

In [6]:
def f_fourier_integrand(u, j, x, alpha):
    i = 1j
    
    mu = Mu[j]
    sigma2 = Sigma[j, j]
    
    alpha_m_iu = alpha - i*u
    
    log_part = -alpha_m_iu * x
    log_part += mu * alpha_m_iu + .5*sigma2*(alpha_m_iu)**2
    
    tmp = numpy.exp(log_part)
    tmp /= (u + i*alpha)**2
    
    return tmp

def f(index, x):
    factor = -.5/numpy.pi
    
    continue_bool = True
    while continue_bool:
        try:
            alpha = 1.5*numpy.random.rand()
            integral = fourier_integral1d(f_fourier_integrand, index, x, alpha)
            continue_bool = False
        except Exception, e:
            print "Not converging for x = %s, alpha = %s" % (x, alpha)
    
    return factor * integral

In [7]:
def g_fourier_integrand(u, j, x, alpha):
    i = 1j
    
    mu = Mu[j]
    sigma2 = Sigma[j, j]
    
    alpha_m_iu = alpha - i*u
    
    log_part = -alpha_m_iu * x
    log_part += mu * alpha_m_iu + .5*sigma2*(alpha_m_iu)**2
    
    tmp = numpy.exp(log_part)    
    tmp /= (i * (u + alpha*i)**3)
    
    return tmp

def g(index, x):
    factor = +1./numpy.pi
    
    continue_bool = True
    while continue_bool:
        try:
            alpha = 1.5*numpy.random.rand(1)
            integral = fourier_integral1d(g_fourier_integrand, index, x, alpha)
            continue_bool = False
        except Exception, e:
            print "Not converging for x = %s, alpha = %s" % (x, alpha)
    
    return factor * integral

In [8]:
def fourier_integral2d(cplx_integrand, j, k, x, y, v_alpha):
    import warnings
    warnings.filterwarnings('error')
    
    bounds = [lambda xx: -numpy.inf, lambda xx: numpy.inf]

    def real_integrand(u, v):
        return scipy.real(cplx_integrand(u, v, j, k, x, y, v_alpha))

    real_quad = dblquad(real_integrand, -numpy.inf, numpy.inf,
                                        bounds[0], bounds[1], epsrel=1e-4)[0]
    
    return real_quad

In [9]:
def h_fourier_integrand(u, v, j, k, x, y, v_alpha):
    mu = numpy.array([Mu[j], Mu[k]])
    sigma2_j, sigma2_k, cross = Sigma[j, j], Sigma[k, k], Sigma[j, k]
    sigma = numpy.array([[sigma2_j, cross],
                         [cross, sigma2_k]])

    i = 1j
    alpha_m_iu = v_alpha - i*numpy.array([u, v])
    
    log_part = numpy.dot(-alpha_m_iu, [x, y])    
    log_part += numpy.dot(mu, alpha_m_iu) + .5 * numpy.dot(alpha_m_iu, numpy.dot(sigma, alpha_m_iu))
    
    tmp = numpy.exp(log_part)
    tmp /= (u + i*v_alpha[0])**2 * (v + i*v_alpha[1])**2
    
    return tmp

def h(j, k, x, y):
    factor = .25 / (numpy.pi**2)
    #x, y = v_x[0], v_x[1]
    
    continue_bool = True
    while continue_bool:
        try:
            alpha = 1.5*numpy.random.rand(2)
            integral = fourier_integral2d(h_fourier_integrand, j, k, x, y, alpha)
            continue_bool = False
        except Exception, e:
            print "Not converging for x, y = %s, %s alpha = %s" % (x, y, alpha)
    
    return factor * integral

In [10]:
def l_fourier_integrand(u, v, j, k, x, y, v_alpha):
    mu = numpy.array([Mu[j], Mu[k]])
    sigma2_j, sigma2_k, cross = Sigma[j, j], Sigma[k, k], Sigma[j, k]
    sigma = numpy.array([[sigma2_j, cross],
                         [cross, sigma2_k]])

    i = 1j
    alpha_m_iu = v_alpha - i*numpy.array([u, v])
    
    log_part = numpy.dot(-alpha_m_iu, [x, y])    
    log_part += numpy.dot(mu, alpha_m_iu) + .5 * numpy.dot(alpha_m_iu, numpy.dot(sigma, alpha_m_iu))
    
    tmp = numpy.exp(log_part)
    tmp *= 1. / (i * (u + i*v_alpha[0])**2 * (v + i*v_alpha[1]))
    
    return tmp

def l(j, k, x, y):
    factor = .25 / (numpy.pi**2)
    #x, y = v_x[0], v_x[1]
    
    continue_bool = True
    while continue_bool:
        try:
            alpha = 1.5*numpy.random.rand(2)
            integral = fourier_integral2d(l_fourier_integrand, j, k, x, y, alpha)
            continue_bool = False
        except Exception, e:
            print "Not converging for x, y = %s, %s alpha = %s" % (x, y, alpha)
    
    return factor * integral

In [11]:
class WrapperFunc(object):
    def __init__(self, func, indexes):
        self._i = indexes
        self._f = func
        
    def __call__(self, *x):
        tmp_args = self._i + list(x)
        return self._f(*tmp_args)

## Put the variables and functions to the remote engines

In [12]:
def init_funcs():
    funcs = dict()
    
    for k in range(N):
        ek = WrapperFunc(e, [k])
        fk = WrapperFunc(f, [k])
        gk = WrapperFunc(g, [k])
    
        hk = dict()
        lk = dict()
        for j in range(N):
            if j < k:
                hjk = WrapperFunc(h, [j, k])
                hk[j] = hjk
                
            if j != k:
                ljk = WrapperFunc(l, [j, k])
                lk[j] = ljk
        
        funcs[k] = {"e": ek, "f": fk, "g": gk, 
                    "hk": hk, "lk": lk}
        
    return funcs

In [13]:
def prepare_parameterized_funcs(dict_funcs):
    res = dict()
        
    for k, v in dict_funcs.iteritems():
        for kk, vv in v.iteritems():
            if isinstance(vv, dict):
                for j, f in vv.iteritems():
                    key = "%s%s%s" % (kk[:-1], j, k)
                    res[key] = (f, j, k)
            else:
                key = "%s%s" % (kk, k)
                res[key] = (vv, k)
        
    return res

In [14]:
def run_par_func(param):
    f = param[0]
    vm = []
    for i_m in param[1:]:
        vm.append(M[i_m])
    
    return f(*vm)

In [15]:
funcs = init_funcs()
flatten_funcs = prepare_parameterized_funcs(funcs)

In [16]:
from timeit import default_timer as timer

In [17]:
from scipy.optimize import fsolve

def optimize(c, **kwargs):
    global N
    
    __results = {key: dict() for key in "efghl"}
        
    def fill_results(lbl, res):
        fname = lbl[0]
        k = int(lbl[-1])

        if len(lbl) == 3:
            j = int(lbl[1])
            if k not in __results[fname]:
                __results[fname][k] = dict()
                
            __results[fname][k][j] = res
        else:
            __results[fname][k] = res
    
    def objective(v_x):
        _lambda = v_x[-1]
        
        # Local version
        global M
        M = numpy.clip(v_x, LOW_BOUND, UPP_BOUND)
        f_res = map(run_par_func, flatten_funcs.values())
        
        for lbl, fx in zip(flatten_funcs.keys(), f_res):
            fill_results(lbl, fx)
            
        last = 0.
        res = numpy.zeros(N + 1)
        
        for k in range(N):
            res_k = 1 + __results["f"][k]
            for j, v in __results["l"][k].iteritems():
                res_k += v
                
            res[k] = (_lambda * res_k - 1.)**2
            
            last += __results["e"][k] + .5 * __results["g"][k]
            if k in __results["h"]:
                for j, v in __results["h"][k].iteritems():
                    last += v
                    
        res[-1] = (last - c)**2
        return res
    
    guess = numpy.zeros(N + 1)
    if 'guess' in kwargs and kwargs['guess'] is not None:
        guess = kwargs.pop('guess')
        
    return fsolve(objective, guess, **kwargs) 

In [18]:
c = 1.

guess = numpy.zeros(N+1)

In [19]:
tic = timer()

continue_ = True
__i = 1
l_N = 0
while continue_:
    print "iteration nb %i" % __i
    optim_result = optimize(c, guess=guess, maxfev=1000, full_output=True)
    continue_ = optim_result[2] != 1
    if continue_:
        guess = optim_result[0]
        __i += 1
        l_N += optim_result[1]["nfev"]
        print
    
toc_m_tic = timer() - tic

print optim_result[-1]

iteration nb 1
Not converging for x, y = -10.0, 10.0 alpha = [ 1.25255863  0.14249414]
Not converging for x, y = -10.0, 10.0 alpha = [ 0.17344892  0.58198033]
Not converging for x, y = -10.0, 10.0 alpha = [ 0.09826251  0.41036934]
Not converging for x, y = -10.0, -10.0 alpha = [ 0.11873348  0.37297498]
Not converging for x, y = -10.0, -10.0 alpha = [ 0.69329662  1.28592258]
Not converging for x, y = -0.156439754327, 0.166927989986 alpha = [ 0.00438346  0.6475389 ]

iteration nb 2

iteration nb 3
The solution converged.


In [20]:
m_star = optim_result[0][:-1]
lb_star = optim_result[0][-1]
cto = toc_m_tic

print "Time of optimization: %.2f sec" % (cto)
print "m star = %s" % m_star
print "lambda star = %s" % lb_star

Time of optimization: 3062.09 sec
m star = [-0.07645816 -0.07645223 -0.05935652]
lambda star = 0.594799355851


In [21]:
l_N = optim_result[1]["nfev"]

print "l(3) = %i" % l_N
print "function evaluated at the output = %s" % optim_result[1]["fvec"]

l(3) = 9
function evaluated at the output = [  4.34336792e-13   2.31819535e-12   6.05831234e-13   1.02161988e-12]


### Just need to copy the results in Excel:

In [22]:
print "%f \n %f \n %f \n %i \n %.2f \n %.2f" % (m_star[0], m_star[2], lb_star, l_N, ctp, cto)

-0.076458 
 -0.059357 
 0.594799 
 9 
 0.00 
 3062.09
